In [1]:
import numpy as np
import pandas as pd
import math
from util import hash_tree
from sklearn.preprocessing import scale

### 所有节点的信息

In [2]:
from htrace import Parse
prefix = "/Users/yangs/Desktop/htrace/wordcount/"

In [3]:
# import os
# files = os.listdir(prefix)
# res = {
#     'always': [],
#     'limit': [],
#     'number': [],
#     'probability': [],
#     'tbucket': [],
#     'never': []
# }
# def clean(name):
#     info = name.split('_')
#     try:
#         _, sampler, _,_, time, th = info
#     except Exception:
#         return
#     res[sampler].extend([time, th[:-4]])
# for file in files:
#     clean(file)
# res = pd.DataFrame(res)

### 计算信息熵

In [155]:
def entropy(s1, s2, prefix):
    src = pd.read_csv(prefix+s1+'.csv')
    t = pd.read_csv(prefix+s2+'.csv')
    proba = np.array([0]*len(src.columns[1:]))
    count = 0
    for idx, name in enumerate(src.columns[1:]):
        if name in t.columns:
            proba[idx] = t[name][0]
            count += proba[idx]
    proba = proba/count
    res = 0.0
    for p in proba:
        if p:
            res -= p * math.log2(p)
    return res

### 计算欧式距离
函数特征值 `<均值(mean), 标准差(std)>`
**缺失函数的话 影响很大** <br>
要把只调用一次的函数剔除掉 因为没啥影响(?) 记得要获取所有节点的信息

In [306]:
def eu_dist(a,s1, s2, prefix):
    src_df = pd.read_csv(prefix+s1+'.csv').drop([0,4,6,3,7,5])
    com_df = pd.read_csv(prefix+s2+'.csv').drop([0,4,6,3,7,5])
    v1, v2 = [], []
    for name in a.names:
        if a.func_feature[name]['count'] < 10:
#             print(a.func_feature[name])
            continue
        v1.append(src_df[name][1])
        v1.append(src_df[name][2])
        if name in com_df.columns:
            v2.append(com_df[name][1])
            v2.append(com_df[name][2])
        else:
            v2.append(0)
            v2.append(0)
        print(name)
        print(v1[-2:])
        print(v2[-2:])
    v1, v2 = np.array(v1), np.array(v2)
    v1, v2 = scale(v1), scale(v2)
    res = np.linalg.norm(v1 - v2)
    return res

In [307]:
a = Parse(prefix+'1.out', True)
hash_tree(a)

21it [00:04,  4.97it/s]
100%|██████████| 127663/127663 [00:00<00:00, 301530.26it/s]


14

In [314]:
p = Parse(prefix+'0.8.out', True)
hash_tree(p)

21it [00:02, 10.08it/s]
100%|██████████| 102306/102306 [00:00<00:00, 276519.00it/s]


13

In [315]:
n = Parse(prefix+'n.out', True)
hash_tree(n)

21it [00:00, 349.81it/s]
100%|██████████| 2168/2168 [00:00<00:00, 291824.49it/s]


14

In [316]:
l = Parse(prefix+'l.out', True)
hash_tree(l)

21it [00:00, 338.71it/s]
100%|██████████| 1918/1918 [00:00<00:00, 295000.92it/s]


14

In [317]:
t = Parse(prefix+'t.out', True)
hash_tree(t)

21it [00:00, 409.00it/s]
100%|██████████| 1416/1416 [00:00<00:00, 321225.29it/s]


13

In [329]:
def allnode():
    res = [a, p, l, n, t]
    for r in res:
        r.build_df(False)

In [330]:
allnode()

In [336]:
eu_dist(a, '1.out', '0.7.out', prefix)

DFSInputStream#byteArrayRead
[0.018043222003929275, 0.5619841959094468]
[0.016450759783047202, 0.1846902757342332]
getFileInfo
[1.270935960591133, 6.350371674695091]
[1.5813953488372092, 7.416933406927066]
DFSClient#rename
[5.9875, 3.2471750137034885]
[5.203703703703703, 3.594438206974715]
listPaths
[1.5476190476190477, 4.52204562469503]
[1.6875, 5.738593173670986]
DistributedFileSystem#create
[7.512195121951219, 3.115461051108745]
[6.16, 1.724335620850342]
DFSOutputStream#close
[29.97560975609756, 13.552652516902453]
[35.42307692307692, 44.55124965872278]


2.0251014660162294

In [332]:
eu_dist(a, '1.out', 'l.out', prefix)

DFSInputStream#byteArrayRead
[0.018043222003929275, 0.5619841959094468]
[0.1514950166112957, 2.6672086692360155]
getFileInfo
[1.270935960591133, 6.350371674695091]
[1.29064039408867, 5.518578978713709]
DFSClient#rename
[5.9875, 3.2471750137034885]
[6.3625, 1.3523888849744854]
listPaths
[1.5476190476190477, 4.52204562469503]
[1.4761904761904765, 3.902707598191319]
DistributedFileSystem#create
[7.512195121951219, 3.115461051108745]
[7.853658536585367, 4.672049740797693]
DFSOutputStream#close
[29.97560975609756, 13.552652516902453]
[27.51219512195122, 11.249715443555699]


0.49825277957539443

In [333]:
eu_dist(a, '1.out', 'n.out', prefix)

DFSInputStream#byteArrayRead
[0.018043222003929275, 0.5619841959094468]
[0.10005685048322913, 0.3020502507701366]
getFileInfo
[1.270935960591133, 6.350371674695091]
[1.3333333333333333, 6.445411804790548]
DFSClient#rename
[5.9875, 3.2471750137034885]
[6.278481012658228, 4.5739976031205805]
listPaths
[1.5476190476190477, 4.52204562469503]
[1.5853658536585369, 4.883521320502745]
DistributedFileSystem#create
[7.512195121951219, 3.115461051108745]
[8.3, 5.084919886921549]
DFSOutputStream#close
[29.97560975609756, 13.552652516902453]
[29.375, 9.582803775781294]


0.5837358439968576

In [334]:
eu_dist(a, '1.out', 't.out', prefix)

DFSInputStream#byteArrayRead
[0.018043222003929275, 0.5619841959094468]
[0.21036889332003988, 3.138297999687058]
getFileInfo
[1.270935960591133, 6.350371674695091]
[1.3645320197044335, 6.340590842374654]
DFSClient#rename
[5.9875, 3.2471750137034885]
[4.3375, 2.24422606546475]
listPaths
[1.5476190476190477, 4.52204562469503]
[1.738095238095238, 4.638437099111704]
DistributedFileSystem#create
[7.512195121951219, 3.115461051108745]
[6.585365853658537, 3.63301259119823]
DFSOutputStream#close
[29.97560975609756, 13.552652516902453]
[24.34146341463415, 12.090512305311053]


0.4417236294632078

In [327]:
pt = p.df['name'].value_counts().to_frame()
at = a.df['name'].value_counts().to_frame()
nt = n.df['name'].value_counts().to_frame()
lt = l.df['name'].value_counts().to_frame()
tt = t.df['name'].value_counts().to_frame()
c = pd.concat([at,pt,nt,lt,tt], axis=1, sort=True)
c.columns = ['always', 'p', 'n', 'l', 't']
c

,always,p,n,l,t
BlockReaderRemote2#readNextPacket,126,100.0,2.0,3,1
ClientNamenodeProtocol#addBlock,40,34.0,39.0,40,40
ClientNamenodeProtocol#complete,41,35.0,40.0,41,41
ClientNamenodeProtocol#create,41,32.0,40.0,41,41
ClientNamenodeProtocol#delete,1,NaN,1.0,1,1
ClientNamenodeProtocol#getBlockLocations,1,1.0,1.0,1,1
ClientNamenodeProtocol#getFileInfo,204,167.0,202.0,204,204
ClientNamenodeProtocol#getListing,42,36.0,41.0,42,42
ClientNamenodeProtocol#getServerDefaults,1,1.0,NaN,1,1
ClientNamenodeProtocol#mkdirs,1,1.0,1.0,1,1


In [328]:
entropy('1.out', '0.7.out', prefix)

0.03974165943633943

In [175]:
entropy('1.out', '0.1.out', prefix)

1.0733655629884318

In [176]:
entropy('1.out', 'n.out', prefix)

2.6605886735693787

In [177]:
entropy('1.out', 'l.out', prefix)

2.823119803318637

In [178]:
entropy('1.out', 't.out', prefix)

3.18372315212511